# Evaluating Trading Strategies with Python 101

## Setup

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import quantstats as qs

# api key
from api_keys import FMP_API_KEY

# plotting settings
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (16, 8)

In [ ]:
%config InlineBackend.figure_format = "retina"

## Downloading data

In [ ]:
SYMBOL = "AAPL"
START_DATE = "2021-01-01"

In [ ]:
def get_price_data(symbol, start_date):
    hist_price_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={start_date}&apikey={FMP_API_KEY}"
    r_json = requests.get(hist_price_url).json()
    df = pd.DataFrame(r_json["historical"]).set_index("date").sort_index()
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
df = get_price_data(SYMBOL, START_DATE)
df

## Simple trading strategy

In [ ]:
df = df[["adjClose"]]
df["log_rtn"] = df["adjClose"].apply(np.log).diff(1)
df["sma_20"] = df["adjClose"].rolling(window=20).mean()
df["position"] = (df["adjClose"] > df["sma_20"]).astype(int)
df

In [ ]:
fig, ax = plt.subplots(2, sharex=True)
df.loc[:, ["adjClose", "sma_20"]].plot(ax=ax[0])
df.loc[:, "position"].plot(ax=ax[1])
ax[0].set_title("Preview of our SMA strategy")

In [ ]:
df["strategy_rtn"] = df["position"].shift(1) * df["log_rtn"]

In [ ]:
df

In [ ]:
qs.reports.metrics(df["strategy_rtn"], benchmark="AAPL", mode="basic")

In [ ]:
qs.reports.html(df["strategy_rtn"], benchmark="AAPL",)